# DeLorean
## im quite tired atm tbh

### 1. Setup workspace

In [1]:
import pandas as pd
import numpy as np
import re

#Append folder for custom libraries
import sys
sys.path.append('libraries')

#Import custom library
import shinypanda

cars_price = pd.read_csv('data/cars_train.csv')

### 2. Explore the dataset

In [2]:
#Head of the dataframe
cars_price.head()

,Id,city,year,manufacturer,make,condition,cylinders,fuel,odometer,title_status,...,paint_color,lat,long,county_fips,county_name,state_fips,state_code,state_name,weather,price
0,559327,baltimore,2006.0,NaN,Scion tc,excellent,4 cylinders,gas,190000.0,clean,...,blue,39.287000,-76.647600,24510.0,Baltimore City,24.0,MD,Maryland,56.0,3200
1,1429566,carbondale,2018.0,dodge,charger sxt,NaN,NaN,gas,NaN,clean,...,NaN,37.720000,-89.215800,17077.0,Jackson,17.0,IL,Illinois,48.0,30620
2,931606,thumb,1997.0,ford,f 250 2 wheel dr pickup,fair,NaN,gas,NaN,clean,...,white,43.483300,-83.383500,26157.0,Tuscola,26.0,MI,Michigan,45.0,1800
3,1265412,laredo,2003.0,ram,NaN,NaN,8 cylinders,gas,NaN,clean,...,NaN,27.850069,-99.668883,48479.0,Webb,48.0,TX,Texas,67.0,4500
4,1133731,ocala,2000.0,dodge,1500,NaN,NaN,gas,NaN,clean,...,NaN,29.165000,-81.539900,12069.0,Lake,12.0,FL,Florida,65.0,1400


In [3]:
cars_price.count()

Id              469992
city            469992
year            468279
manufacturer    432732
make            450914
condition       278971
cylinders       281916
fuel            467148
odometer        316527
title_status    469284
transmission    467545
drive           289658
size            163217
type            278352
paint_color     280379
lat             469992
long            469992
county_fips     453983
county_name     453983
state_fips      453983
state_code      453983
state_name      469992
weather         453810
price           469992
dtype: int64

In [4]:
#Type of data
#cars_price.dtypes

In [5]:
#Descriptive stats
#cars_price.describe()

### 3. Data cleaning
#### In order to use sklearn algorithms, the entire dataset must contain operable numerical data

#### 3.1 Setup dataframe

In [6]:
#Remove all location info but State Code (could affect to taxes)
location_columns = ['city', 'lat', 'long', 'county_fips', 'county_name', 'state_fips', 'state_name']
cars_price = cars_price.drop(columns=location_columns, axis=1)

#Remove unpopulated columns
cars_price = cars_price.drop(columns=['drive', 'size', 'type', 'paint_color', 'condition', 'cylinders', 'odometer'], axis=1)

#Rename columns
cars_price.rename(columns={'manufacturer':'brand',
                          'make':'model',
                          'state_code':'state'}, 
                          inplace=True)

cars_price.head()

,year,brand,model,fuel,title_status,transmission,state,weather,price
0,2006.0,NaN,Scion tc,gas,clean,automatic,MD,56.0,3200
1,2018.0,dodge,charger sxt,gas,clean,automatic,IL,48.0,30620
2,1997.0,ford,f 250 2 wheel dr pickup,gas,clean,automatic,MI,45.0,1800
3,2003.0,ram,NaN,gas,clean,automatic,TX,67.0,4500
4,2000.0,dodge,1500,gas,clean,automatic,FL,65.0,1400


#### 3.2 Clean 'brand' column

In [7]:
#Transform column type from 'object' to string
cars_price['brand'] = cars_price['brand'].astype('str') 

brands = sorted(cars_price.brand.unique())
print('Original brands list:', brands)

Original brands list: ['acura', 'alfa', 'alfa-romeo', 'aston', 'aston-martin', 'audi', 'bmw', 'buick', 'cadillac', 'chev', 'chevrolet', 'chevy', 'chrysler', 'datsun', 'dodge', 'ferrari', 'fiat', 'ford', 'gmc', 'harley', 'harley-davidson', 'honda', 'hyundai', 'infiniti', 'infinity', 'jaguar', 'jeep', 'kia', 'land rover', 'landrover', 'lexus', 'lincoln', 'mazda', 'mercedes', 'mercedes-benz', 'mercury', 'mini', 'mitsubishi', 'morgan', 'nan', 'nissan', 'pontiac', 'porche', 'ram', 'rover', 'saturn', 'subaru', 'toyota', 'volkswagen', 'volvo', 'vw']


In [8]:
#Remove NaN
cars_price['brand'] = cars_price.brand.replace({r'nan': ''}, regex=True)

#Fix duplicity: 'alfa' and 'alfa-romeo'
cars_price['brand'] = cars_price.brand.replace({r'alfa-romeo': 'alfa'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'alfa': 'alfa-romeo'}, regex=True)

#Fix duplicity: 'aston' and 'aston-martin'
cars_price['brand'] = cars_price.brand.replace({r'aston-martin': 'aston'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'aston': 'aston-martin'}, regex=True)

#Fix duplicity: 'chev', 'chevy' and 'aston-martin'
cars_price['brand'] = cars_price.brand.replace({r'chevrolet': 'chev'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'chevy': 'chev'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'chev': 'chevrolet'}, regex=True)

#Fix duplicity: 'harley' and 'harley-davidson'
cars_price['brand'] = cars_price.brand.replace({r'harley-davidson': 'harley'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'harley': 'harley-davidson'}, regex=True)

#Fix misspelling: 'infinity' instead of 'infinity'
cars_price['brand'] = cars_price.brand.replace({r'infinity': 'infiniti'}, regex=True)

#Fix duplicity: 'rover', 'landrover' and 'land rover'
cars_price['brand'] = cars_price.brand.replace({r'landrover': 'land-rovex'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'land rover': 'land-rovex'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'rover': 'land-rovex'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'land-rovex': 'land-rover'}, regex=True)

#Fix duplicity: 'landrover' and 'land rover'
cars_price['brand'] = cars_price.brand.replace({r'mercedes-benz': 'mercedes'}, regex=True)
cars_price['brand'] = cars_price.brand.replace({r'mercedes': 'mercedes-benz'}, regex=True)

#Fix misspelling: 'porsche' instead of 'porche'
cars_price['brand'] = cars_price.brand.replace({r'porche': 'porsche'}, regex=True)

#Fix duplicity: 'vw' and 'volkswagen'
cars_price['brand'] = cars_price.brand.replace({r'vw': 'volkswagen'}, regex=True)

brands = sorted(cars_price.brand.unique())
print('Clean brands list:', brands)

Clean brands list: ['', 'acura', 'alfa-romeo', 'aston-martin', 'audi', 'bmw', 'buick', 'cadillac', 'chevrolet', 'chrysler', 'datsun', 'dodge', 'ferrari', 'fiat', 'ford', 'gmc', 'harley-davidson', 'honda', 'hyundai', 'infiniti', 'jaguar', 'jeep', 'kia', 'land-rover', 'lexus', 'lincoln', 'mazda', 'mercedes-benz', 'mercury', 'mini', 'mitsubishi', 'morgan', 'nissan', 'pontiac', 'porsche', 'ram', 'saturn', 'subaru', 'toyota', 'volkswagen', 'volvo']


In [9]:
#Import new brands (scraped from Wikipedia using brands-importer.ipynb)
new_brands = pd.read_csv('data/brands.csv')
new_brands.head()

,Unnamed: 0,name
0,0,adelmo
1,1,asa
2,2,de-carlo
3,3,feresa
4,4,koller


In [10]:
#Convert it into a list and merge both lists
new_brands = list(new_brands['name'])
brands = list(set(brands + new_brands))

#### 3.3 Clean 'model' column

In [11]:
print('There are', len(cars_price['model'].unique()), 'different models in the first dataset')

There are 46593 different models in the first dataset


In [12]:
#Transform column type from 'object' to string
cars_price['model'] = cars_price['model'].astype('str')

#Make it lowercase
cars_price['model'] = cars_price['model'].str.lower()

cars_price.head()

,year,brand,model,fuel,title_status,transmission,state,weather,price
0,2006.0,,scion tc,gas,clean,automatic,MD,56.0,3200
1,2018.0,dodge,charger sxt,gas,clean,automatic,IL,48.0,30620
2,1997.0,ford,f 250 2 wheel dr pickup,gas,clean,automatic,MI,45.0,1800
3,2003.0,ram,nan,gas,clean,automatic,TX,67.0,4500
4,2000.0,dodge,1500,gas,clean,automatic,FL,65.0,1400


In [13]:
#Fix misspelling
cars_price['model'] = cars_price.model.replace({r'chervolet': 'chevrolet'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'volksvagen': 'volkswagen'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'mercedes-benzbenz': 'mercedes-benz'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'nissaan': 'nissan'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'totota': 'toyota'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'suburu': 'subaru'}, regex=True)
cars_price['model'] = cars_price.model.replace({r'crystler': 'chrysler'}, regex=True)

In [14]:
#Split 'model' data
cars_price['extracted_brand'] = cars_price["model"].str.split(" ", n=-1, expand=False) 

#When 'model' contains a string that already exists in 'brands', export it in a new column
def brand_in_model(row):
    
    brand = list(set(row['extracted_brand']).intersection(brands))
    return brand

#Apply the function and convert column to string again
cars_price['extracted_brand'] = cars_price.apply(lambda row: brand_in_model(row), axis=1)
cars_price['extracted_brand'] = cars_price['extracted_brand'].apply(lambda brand: ','.join(map(str, brand)))

cars_price.head()

,year,brand,model,fuel,title_status,transmission,state,weather,price,extracted_brand
0,2006.0,,scion tc,gas,clean,automatic,MD,56.0,3200,scion
1,2018.0,dodge,charger sxt,gas,clean,automatic,IL,48.0,30620,
2,1997.0,ford,f 250 2 wheel dr pickup,gas,clean,automatic,MI,45.0,1800,dr
3,2003.0,ram,nan,gas,clean,automatic,TX,67.0,4500,
4,2000.0,dodge,1500,gas,clean,automatic,FL,65.0,1400,


In [15]:
#Split 'extracted brand' to preserve only the first match with brands list
split_brand = cars_price['extracted_brand'].str.split(',', n=1, expand=True) 
cars_price['extracted_brand'] = split_brand[0]

print('New brands found in \'model\' column:', sorted(cars_price['extracted_brand'].unique()))

New brands found in 'model' column: ['', 'abarth', 'ac', 'aero', 'alpine', 'amc', 'astra', 'auburn', 'austin', 'austin-healey', 'bentley', 'berkeley', 'bertone', 'blade', 'borgward', 'bricklin', 'bugatti', 'bullet', 'cap', 'checker', 'cheetah', 'chevrolet', 'chevron', 'chrysler', 'cla', 'cmc', 'cobra', 'colt', 'cord', 'crosley', 'daewoo', 'daf', 'daihatsu', 'daimler', 'daytona', 'desoto', 'direct', 'dr', 'ds', 'e-z-go', 'eagle', 'edsel', 'enfield', 'envoy', 'fab', 'falcon', 'frazer', 'gaz', 'genesis', 'geo', 'glas', 'goggomobil', 'gordon', 'healey', 'hillman', 'hino', 'hudson', 'hummer', 'imperial', 'invicta', 'isuzu', 'jensen', 'jowett', 'kaiser', 'karma', 'kawasaki', 'lada', 'lafayette', 'lamborghini', 'lancia', 'lasalle', 'lloyd', 'lotus', 'marauder', 'marquette', 'maserati', 'master', 'maxwell', 'maybach', 'mclaren', 'mclaughlin', 'mercedes-amg', 'merkur', 'meteor', 'metropolitan', 'mg', 'mine', 'minerva', 'monarch', 'moon', 'morris', 'mustang', 'nash', 'nissan', 'nsu', 'oldsmobile

In [16]:
cars_price.head()

,year,brand,model,fuel,title_status,transmission,state,weather,price,extracted_brand
0,2006.0,,scion tc,gas,clean,automatic,MD,56.0,3200,scion
1,2018.0,dodge,charger sxt,gas,clean,automatic,IL,48.0,30620,
2,1997.0,ford,f 250 2 wheel dr pickup,gas,clean,automatic,MI,45.0,1800,dr
3,2003.0,ram,nan,gas,clean,automatic,TX,67.0,4500,
4,2000.0,dodge,1500,gas,clean,automatic,FL,65.0,1400,


In [17]:
#Where 'brand' is not available, use 'extracted_brand'
def complete_brand(row):
    
    brand_doesnt_exist = row['brand']==''
    extracted_brand_exists = row['extracted_brand']!=''
    
    if brand_doesnt_exist and extracted_brand_exists:
        return row['extracted_brand']
    else:
        return row['brand']

cars_price['brand'] = cars_price.apply(lambda row: complete_brand(row), axis=1)

#Remove 'extracted_brand' and 'model' columns
cars_price = cars_price.drop(['model', 'extracted_brand'], axis=1)

In [18]:
cars_price.head()

,year,brand,fuel,title_status,transmission,state,weather,price
0,2006.0,scion,gas,clean,automatic,MD,56.0,3200
1,2018.0,dodge,gas,clean,automatic,IL,48.0,30620
2,1997.0,ford,gas,clean,automatic,MI,45.0,1800
3,2003.0,ram,gas,clean,automatic,TX,67.0,4500
4,2000.0,dodge,gas,clean,automatic,FL,65.0,1400


### 3. Make model

In [19]:
cars_price.dtypes

year            float64
brand            object
fuel             object
title_status     object
transmission     object
state            object
weather         float64
price             int64
dtype: object

In [20]:
cars_price['brand'] = cars_price['brand'].astype('str')
cars_price['fuel'] = cars_price['fuel'].astype('str') 
cars_price['title_status'] = cars_price['title_status'].astype('str') 
cars_price['transmission'] = cars_price['transmission'].astype('str') 
cars_price['state'] = cars_price['state'].astype('str') 

In [21]:
cars_price.dtypes

year            float64
brand            object
fuel             object
title_status     object
transmission     object
state            object
weather         float64
price             int64
dtype: object

In [22]:
year_values = sorted(cars_price['year'].unique())
brand_values = sorted(cars_price['brand'].unique())
fuel_values = cars_price['fuel'].unique()
title_status = sorted(cars_price['title_status'].unique())
transmission_values = sorted(cars_price['transmission'].unique())
state_values = sorted(cars_price['state'].unique())
weather_values = sorted(cars_price['weather'].unique())
price_values = sorted(cars_price['price'].unique())

In [23]:
cars_price = cars_price.replace(r'^\s*$', np.nan, regex=True)

In [24]:
cars_price = cars_price.dropna()

In [25]:
cars_price.count()

year            426861
brand           426861
fuel            426861
title_status    426861
transmission    426861
state           426861
weather         426861
price           426861
dtype: int64

In [26]:
final_dataset = pd.get_dummies(cars_price)

In [27]:
final_dataset.head()

,year,weather,price,brand_ac,brand_acura,brand_aero,brand_alfa-romeo,brand_alpine,brand_amc,brand_aston-martin,...,state_SD,state_TN,state_TX,state_UT,state_VA,state_VT,state_WA,state_WI,state_WV,state_WY
0,2006.0,56.0,3200,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2018.0,48.0,30620,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1997.0,45.0,1800,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2003.0,67.0,4500,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,2000.0,65.0,1400,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
#only_brands = cars_price.drop('state', axis=1)
only_brands = pd.get_dummies(only_brands)

In [29]:
cars_price.count()

year            426861
brand           426861
fuel            426861
title_status    426861
transmission    426861
state           426861
weather         426861
price           426861
dtype: int64

In [30]:
only_brands.head()

,year,weather,price,brand_ac,brand_acura,brand_aero,brand_alfa-romeo,brand_alpine,brand_amc,brand_aston-martin,...,title_status_lien,title_status_missing,title_status_nan,title_status_parts only,title_status_rebuilt,title_status_salvage,transmission_automatic,transmission_manual,transmission_nan,transmission_other
0,2006.0,56.0,3200,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,2018.0,48.0,30620,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,1997.0,45.0,1800,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,2003.0,67.0,4500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,2000.0,65.0,1400,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [32]:
only_brands.to_csv(r'data/trained_dataframe.csv', index=False)